In [34]:
with open('texts/AEI_Brecko.txt', 'r') as file:
    text = file.read()

In [44]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="mistral"
)

In [35]:
for index, row in df.iterrows():
    prompt = row['Prompt'] + text
    generated_post = llm.invoke(prompt)
    df.at[index, 'Output_LLama3_70b_with_references'] = generated_post

In [36]:
df

,ID,Prompt,Output_Llama3_8b,Output_Llama3_70b,Output_Mistral,Output_Mistral_with_references,Output_LLama3_70b_with_references
0,1,Can you create me a LinkedIn post blog from su...,Here's a LinkedIn post summarizing the text:\n...,Here's a possible LinkedIn post based on the p...,It appears you have provided a summary of a r...,It appears that the provided text is a resear...,Here's a LinkedIn post based on the provided t...
1,2,Can you create me a LinkedIn post blog from su...,Here's a LinkedIn post based on the research a...,Here's a LinkedIn post summarizing the researc...,The paper presents the development of a web a...,The text appears to be a research paper abstr...,Here's a potential LinkedIn post summarizing t...
2,3,Summarize this research article into a brief L...,Here's a brief LinkedIn post summarizing the r...,Here's a brief LinkedIn post summarizing the r...,"Based on the information provided, it appears...",Here is a summary of the information provided...,Here's a brief LinkedIn post summarizing the r...
3,4,Draft a LinkedIn post summarizing the key poin...,Here's a LinkedIn post summarizing the key poi...,Here's a draft LinkedIn post summarizing the k...,The paper discusses the development of a web ...,The text provided appears to be a research pa...,Here's a LinkedIn post summarizing the key poi...
4,5,Create a LinkedIn post summarizing this report...,Here's a LinkedIn post summarizing the report ...,Here's a LinkedIn post summarizing the report:...,"Based on the information provided, the author...",It appears you have provided a list of resour...,Here's a LinkedIn post summarizing the report:...
5,6,Generate a LinkedIn post summarizing this stud...,Here's a summary LinkedIn post:\n\n**Breaking ...,Here's a LinkedIn post summarizing the study f...,"Based on the information provided, the follow...",This text appears to be a combination of rese...,Here's a LinkedIn post summarizing the study:\...
6,7,Write a LinkedIn post summarizing this paper f...,Here's a summary of the paper for a technical ...,Here's a LinkedIn post summarizing the paper f...,"In conclusion, the research presented in this...",It appears you have provided a list of resour...,Here's a LinkedIn post summarizing the paper f...
7,8,Create a LinkedIn post with a brief overview o...,Here's a LinkedIn post based on the provided c...,Here's a possible LinkedIn post based on the p...,Based on the information provided in the text...,It appears you have provided a list of refere...,Here's a potential LinkedIn post based on the ...
8,9,Draft a LinkedIn post with a detailed analysis...,Here's a draft LinkedIn post analyzing the art...,Here's a draft LinkedIn post with a detailed a...,"In this paper, the authors describe the devel...",It appears you have provided a list of refere...,Here's a draft LinkedIn post with a detailed a...
9,10,Summarize this article in basic English for Li...,Here's a summary of the article in basic Engli...,Here's a summary of the article in basic Engli...,This paper discusses the development of a web...,This text appears to be a scholarly article o...,Here's a summary of the article in basic Engli...


In [27]:
df.to_csv(csv_file_path, index=False)

In [ ]:
# Generate output to new prompts
for index, row in df.iterrows():
    if pd.isnull(row['Output_Llama3']):
        prompt = row['Prompt'] + text
        generated_post = llm.invoke(prompt)
        df.at[index, 'Output_Llama3'] = generated_post

df.to_csv(csv_file_path, index=False)

In [ ]:
row = df[df['ID'] == 26].iloc[0]
prompt = row['Prompt'] + text

#print("Prompt:\n", prompt)
print("\nGenerated Output:\n", generated_post)

In [42]:
df = pd.read_csv('csv/MergedPromptsAndSummaries.csv')

df['Modified Prompt'] = df.apply(lambda row: row['Prompt'].replace("{text}", row['Summarized Text']), axis=1)

selected_prompt = df['Modified Prompt'].iloc[15]

In [45]:
%%timeit

generated_post = llm.invoke(selected_prompt)


5.42 s ± 1.94 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def generate_linkedin_post(summary_text):
    prompt = f"""Create a LinkedIn post:
    - summarize the text {summary_text}
    - use emoticons
    - put hashtags at the end
    - aimed at a wide audience
    - have a formal tone
    - use perspective 'first person plural'
    - use link to original paper 'paper.com/article'"""

    post = llm.invoke(prompt)
    return post

csv_file_path = 'csv/SummaryText.csv'
df_summary = pd.read_csv(csv_file_path)

output_data = {
    "ID": [],
    "Prompt": [],
    "Summarized Text": [],
    "LinkedIn Post": []
}

for index, row in df_summary.iterrows():
    summarized_text = row['Summarized Text']
    post = generate_linkedin_post(summarized_text)
    output_data["ID"].append(index + 1) 
    output_data["Prompt"].append("""Create a LinkedIn post:
    - summarize the text {summary_text}
    - use emoticons
    - put hashtags at the end
    - aimed at a wide audience
    - have a formal tone
    - use perspective 'first person plural'
    - use link to original paper 'paper.com/article'""")
    output_data["Summarized Text"].append(summarized_text)
    output_data["LinkedIn Post"].append(post)

df_output = pd.DataFrame(output_data)

output_csv_path = 'csv/OutputSummarized.csv'
df_output.to_csv(output_csv_path, index=False)

print("Output summarized data saved successfully.")


In [ ]:
from langchain_community.llms import Ollama
import pandas as pd

df = pd.read_csv('csv/MergedNewPromptsAndAbstract.csv') 

df['Output_Llama3_8b'] = ''
df['Output_Llama3_70b'] = ''
df['Output_Mistral'] = ''

def generate_outputs(row):
    modified_prompt = row['Prompt'].replace("{text}", row['Summarized Text'])
    
    # Llama3 8b model
    llm = Ollama(model="llama3")
    try:
        output = llm.invoke(modified_prompt)
        row['Output_Llama3_8b'] = output
    except Exception as e:
        row['Output_Llama3_8b'] = str(e)
    
    # Llama3 70b model
    llm = Ollama(model="llama3:70b")
    try:
        output = llm.invoke(modified_prompt)
        row['Output_Llama3_70b'] = output
    except Exception as e:
        row['Output_Llama3_70b'] = str(e)
    
    # Mistral model
    llm = Ollama(model="mistral")
    try:
        output = llm.invoke(modified_prompt)
        row['Output_Mistral'] = output
    except Exception as e:
        row['Output_Mistral'] = str(e)
    
    return row

df = df.apply(generate_outputs, axis=1)

df.to_csv('csv/NewPromptsAndAbstractOutput.csv', index=False)